In [1]:
import pymysql
import pandas as pd
import time
import numpy as np
import random
import os
import sys

import codecs
import jieba
import pkuseg
import math
import pickle
import gensim
import stanfordnlp as stanfordnlp
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim.models.word2vec import LineSentence
from sklearn.manifold import TSNE
import re, sys, codecs
import hanziconv
from hanziconv import HanziConv
import pytest
import re
import nltk
from nltk.tree import Tree
import matplotlib.pyplot as plt
%matplotlib inline

from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
from nltk.tokenize.stanford_segmenter import StanfordSegmenter
import stanfordcorenlp
import stanfordnlp
from stanfordcorenlp import StanfordCoreNLP

from functools import wraps
from collections import defaultdict
from collections import Counter
from functools import lru_cache

In [2]:
file_name = "/Users/lingrowzhang/Documents/GitHub/Artificial-Intelligence-for-NLP/"

In [3]:
headname = ["id","author","source","content","feature","title","url"]
news_chinese = pd.read_csv(file_name + "date/news_chinese.csv")
news_chinese.columns = headname

In [13]:
news_chinese_try = news_chinese[:10]

In [16]:
news_chinese_try.feature[0]

'{"type":"科技","site":"cnbeta","commentNum":"37","joinNum":"20007","clickNum":"19920","shareNum":"0","url":"http://www.cnbeta.com/articles/tech/623597.htm","site_sign":"cnbeta","label":"","summary":"MIUI 9或许真的要来了！日前，微博数码博主@罗逸文V曝光了MIUI9首批支持机型，共有15款机型在列。具体包括小米6、小米Note2、小米MIX、小米5c、小米5s、小米5s Plus、小米5、小米4s、小米4c、小米Max 2、小米Max 64GB（128GB）、小米Max 32GB、小米Note顶配版、红米Note 4X(高通版)、红米4X手机。","is_hot":"2","imgUrls":"http://static.cnbetacdn.com/thumb/mini/article/2017/0618/188676856afc70a.jpg"}'

In [5]:
def clear_content(string_list):
    result = []
    for string in string_list:
        the_str  = ""
        if str(string) == 'nan':
            continue
        for the_chat in str(string):
            if ('\u4e00' <= the_chat <= '\u9fa5'):
                j_label = 0
                the_str += the_chat
            elif the_chat in ["，",":","：","\"","“","”","？","！","(",")","《","》","、",".",".","",",","%",";","%","【","】","…"]:
                the_str += the_chat
            elif the_chat == "。":
                the_str += the_chat
                j_label = 1
            elif the_chat.isdigit() or the_chat.isalpha():
                the_str += the_chat
            elif the_chat == "+":
                the_str += "、"
            elif the_chat in ["\n","@","\n","/"] and j_label == 0:
                the_str += "。"
            elif the_str == "~":
                the_str += "到"
        result.append(the_str)
    return result
data_content = clear_content(list(news_chinese["content"]))
data_content = data_content[:100]

In [6]:
all_source = []
for the_tmp in list(news_chinese["source"]):
    for the_str in str(the_tmp).split('@'):
        if "http" not in the_str and the_str != "":
            if "第" in the_str:
                try:
                    the_str_i = the_str.index("第")
                except:
                    continue
                if the_str[:the_str_i] not in all_source:
                    all_source.append(the_str[:the_str_i])
            else:
                if the_str not in all_source:
                    all_source.append(the_str)
all_source    

['快科技',
 '新华社',
 '深圳大件事',
 '中国证券报?中证网',
 '威锋网',
 '荆楚网-楚天都市报',
 '证券时报网',
 '中国新闻网',
 '国际在线',
 '中国证券网',
 'solidot',
 '新华网',
 '泡泡网',
 '凤凰体育',
 '雷锋网',
 '中超球评',
 '中国经济周刊',
 '人民日报',
 '央广网',
 '体育君',
 '新浪体育',
 '广州日报',
 'TechWeb',
 '足球小志',
 '人民网',
 '时代财经',
 '新疆日报',
 '中关村在线',
 '网易科技',
 '封面新闻',
 '央视新闻',
 '澎湃新闻网',
 '中青在线',
 '央视新闻客户端',
 '千龙网',
 '新浪综合',
 '中新网',
 'GamerSky游民星空',
 '中国经济网',
 'cnBeta.COM',
 '热点体育',
 '枪炮篮球PiU',
 '中国网财经',
 '知客数码',
 '于斌',
 '霍寿喜',
 '网易科学人',
 '看看新闻KNEWS',
 '最后一公里',
 '伊吾零距离',
 '环球时报-环球网',
 '经济日报',
 '足球部落国际版',
 '参考消息网',
 '黑龙江日报',
 '解放军报',
 '环球网',
 '星城古龙',
 '凤凰网科技',
 '每日经济新闻',
 '新浪科技',
 '经济参考报',
 '三十年莱斯特城球',
 '中国科技网',
 '柳州新闻网',
 '新京报即时新闻',
 'CCTV-央视网',
 '新闻晨报',
 '北青网-北京青年报',
 'linkgou.com',
 '中华网投资',
 '环球科技',
 '北京旅游网',
 '检察日报',
 '海南日报',
 '南方周末',
 '中国台湾网',
 '南宁晚报',
 '时刻体育',
 '北京晨报',
 '凤凰娱乐',
 '球娱天下',
 '中国娱乐网',
 '云南网',
 '中国时尚在线',
 '足球喜剧之王',
 '南国都市报',
 '剥洋葱people',
 '安卓中国',
 '中国青年报',
 '中国网',
 '网络',
 '深读',
 '哎，听说',
 '阳郊网警',
 '燕赵晚报',
 '证券日报',
 '重案组37号',
 '郑州晚报',
 '华尔街见闻',
 'Wind资讯',


In [7]:
all_author = defaultdict(int)
for the_tmp in list(news_chinese["author"]):
    the_str = str(the_tmp)
    for the_str_1 in the_str.split('\u3000'):
        for the_str_2 in the_str_1.split('、'):
            if "http" not in the_str and the_str != "" and the_str != "nan" and the_str not in all_author:
                all_author[the_str_2] += 1     
all_author

defaultdict(int,
            {'胡淑丽_MN7479': 1,
             '张怡': 1,
             '申玉彬 整理': 1,
             '李杭_BJS4645': 1,
             '吴瞬': 1,
             '费天元': 1,
             '周佳 整理': 1,
             '屠震林 PS040': 1,
             '李天奕_NN7528': 1,
             '王璐 PS039': 1,
             '官平': 1,
             '荀建国_NN7379': 1,
             '张田宇': 1,
             '王志胜': 1,
             '查希': 1,
             '陈晨1': 1,
             '徐俊 PS001': 1,
             '李文瑶': 1,
             '郭萍_B7442': 1,
             '乔敬_NN6607': 1,
             '邵小波_NN2479': 1,
             '范姜国一_NN9138': 1,
             '刘效武_NN4113': 1,
             '李晓丹': 1,
             '赵亚萍_NN9005': 1,
             '范立枢': 1,
             '程钢_MN7377': 1,
             '张海桐_NN9053': 1,
             '周楚梦': 1,
             '谷莹_NN6577': 1,
             '仝贺_NN4951': 1,
             '杨秋子_NX4687': 1,
             '官平_NN4892': 1,
             '王辉': 1,
             '李良': 1,
             '黄淑慧': 1,
             '实习记者 张焕昀': 1,
      

In [6]:
model = Word2Vec.load(file_name +'model/word2vec_wiki_200.model')

In [7]:
model_new = Word2Vec.load(file_name +'model/word2vec_wiki_new_200.model')

### 对文章进行句法分析，观察规律（找10句话）

In [18]:
zh_model = StanfordCoreNLP(file_name+r"project_1/stanford-corenlp-full-2018-10-05", port=999, lang='zh')

In [9]:
the_content = data_content[0]
the_content

'此外，自本周6月12日起，除小米手机6等15款机型外，其余机型已暂停更新发布含开发版。体验版内测，稳定版暂不受影响，以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI9的研发之中。MIUI8去年5月发布，距今已有一年有余，也是时候更新换代了。当然，关于MIUI9的确切信息，我们还是等待官方消息。'

In [10]:
# 分词
print('Tokenize:', zh_model.word_tokenize(the_content))

Tokenize: ['此外', '，', '自', '本', '周', '6月', '12日', '起', '，', '除', '小米', '手机', '6', '等', '15', '款', '机型', '外', '，', '其余', '机型', '已', '暂停', '更新', '发布', '含', '开发', '版', '。', '体验', '版', '内测', '，', '稳', '定版', '暂', '不', '受', '影响', '，', '以', '确保', '工程师', '可以', '集中', '全部', '精力', '进行', '系统', '优化', '工作', '。', '有', '人', '猜测', '这', '也', '是', '将', '精力', '主要', '用到', 'MIUI9', '的', '研发', '之中', '。', 'MIUI8', '去年', '5月', '发布', '，', '距今', '已', '有', '一', '年', '有余', '，', '也', '是', '时候', '更新', '换代', '了', '。', '当然', '，', '关于', 'MIUI9', '的', '确切', '信息', '，', '我们', '还是', '等待', '官方', '消息', '。']


In [11]:
# 句子成分标签
print('Part of Speech:', zh_model.pos_tag(the_content))

Part of Speech: [('此外', 'AD'), ('，', 'PU'), ('自', 'P'), ('本', 'DT'), ('周', 'M'), ('6月', 'NT'), ('12日', 'NT'), ('起', 'LC'), ('，', 'PU'), ('除', 'P'), ('小米', 'NN'), ('手机', 'NN'), ('6', 'CD'), ('等', 'M'), ('15', 'CD'), ('款', 'M'), ('机型', 'NN'), ('外', 'LC'), ('，', 'PU'), ('其余', 'DT'), ('机型', 'NN'), ('已', 'AD'), ('暂停', 'VV'), ('更新', 'NN'), ('发布', 'VV'), ('含', 'VV'), ('开发', 'VV'), ('版', 'NN'), ('。', 'PU'), ('体验', 'NN'), ('版', 'NN'), ('内测', 'NN'), ('，', 'PU'), ('稳', 'AD'), ('定版', 'VV'), ('暂', 'AD'), ('不', 'AD'), ('受', 'VV'), ('影响', 'NN'), ('，', 'PU'), ('以', 'MSP'), ('确保', 'VV'), ('工程师', 'NN'), ('可以', 'VV'), ('集中', 'VV'), ('全部', 'DT'), ('精力', 'NN'), ('进行', 'VV'), ('系统', 'NN'), ('优化', 'NN'), ('工作', 'NN'), ('。', 'PU'), ('有', 'VE'), ('人', 'NN'), ('猜测', 'VV'), ('这', 'PN'), ('也', 'AD'), ('是', 'VC'), ('将', 'BA'), ('精力', 'NN'), ('主要', 'JJ'), ('用到', 'VV'), ('MIUI9', 'NR'), ('的', 'DEG'), ('研发', 'NN'), ('之中', 'LC'), ('。', 'PU'), ('MIUI8', 'NR'), ('去年', 'NT'), ('5月', 'NT'), ('发布', 'VV'), ('，', 'PU'), ('距今

In [12]:
# 句法依存分析
print('Part of Speech:',zh_model.dependency_parse(str(the_content)))

Part of Speech: [('ROOT', 0, 23), ('advmod', 23, 1), ('punct', 23, 2), ('case', 7, 3), ('det', 7, 4), ('mark:clf', 4, 5), ('compound:nn', 7, 6), ('nmod:prep', 23, 7), ('case', 7, 8), ('punct', 23, 9), ('case', 17, 10), ('compound:nn', 12, 11), ('compound:nn', 17, 12), ('nummod', 17, 13), ('mark:clf', 13, 14), ('nummod', 17, 15), ('mark:clf', 15, 16), ('nmod:prep', 23, 17), ('case', 17, 18), ('punct', 23, 19), ('det', 21, 20), ('nsubj', 23, 21), ('advmod', 23, 22), ('dobj', 23, 24), ('conj', 23, 25), ('ccomp', 25, 26), ('ccomp', 26, 27), ('dobj', 27, 28), ('punct', 23, 29), ('compound:nn', 31, 30), ('compound:nn', 32, 31), ('nsubj', 35, 32), ('punct', 35, 33), ('advmod', 35, 34), ('conj', 23, 35), ('advmod', 38, 36), ('neg', 38, 37), ('ccomp', 35, 38), ('dobj', 38, 39), ('punct', 38, 40), ('aux:prtmod', 42, 41), ('conj', 38, 42), ('nsubj', 45, 43), ('aux:modal', 45, 44), ('ccomp', 42, 45), ('det', 47, 46), ('dobj', 45, 47), ('conj', 45, 48), ('compound:nn', 51, 49), ('compound:nn', 51, 

In [36]:
str(int(i/100))

'2'

In [19]:
def get_pos_tag(data_content):    
    content_pos_tag = []
    content_pos_tag_i = []
    for i, the_content in enumerate(data_content):
        for the_str in the_content.split("。"):
            if the_str in ["", "nan"]:
                continue
            try:
                content_pos_tag.append(zh_model.pos_tag(str(the_str)))
                content_pos_tag_i.append(i)
            except:
                print(the_content)
                continue
    return content_pos_tag,content_pos_tag_i

In [20]:
def get_parse(data_content):    
    content_parse = []
    content_parse_i = []
    for i, the_content in enumerate(data_content):
        for the_str in the_content.split("。"):
            if the_str in ["", "nan"]:
                continue
            try:
                content_parse.append(zh_model.parse(str(the_str)))
                content_parse_i.append(i)
            except:
                print(the_content)
                continue
    return content_parse,content_parse_i

In [21]:
def get_dependency_parse(data_content):    
    content_dependency_parse = []
    content_dependency_parse_i = []
    for i, the_content in enumerate(data_content):
        for the_str in the_content.split("。"):
            if the_str in ["", "nan"]:
                continue
            try:
                content_dependency_parse.append(zh_model.dependency_parse(str(the_str)))
                content_dependency_parse_i.append(i)
            except:
                print(the_content)
        #         zh_model.close()
                continue
    return content_dependency_parse,content_dependency_parse_i

In [23]:
content_pos_tag,content_pos_tag_i = get_pos_tag(data_content)
content_parse,content_parse_i = get_parse(data_content)
content_dependency_parse,content_dependency_parse_i = get_dependency_parse(data_content)

In [19]:
the_content_j = the_content.split("。")
the_content_j

['此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作',
 '有人猜测这也是将精力主要用到MIUI 9的研发之中',
 '\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了',
 '\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息',
 '\r\n']

In [25]:
content_pos_tag[1]

[('体验', 'NN'),
 ('版', 'NN'),
 ('内测', 'NN'),
 ('，', 'PU'),
 ('稳', 'AD'),
 ('定版', 'VV'),
 ('暂', 'AD'),
 ('不', 'AD'),
 ('受', 'VV'),
 ('影响', 'NN'),
 ('，', 'PU'),
 ('以', 'MSP'),
 ('确保', 'VV'),
 ('工程师', 'NN'),
 ('可以', 'VV'),
 ('集中', 'VV'),
 ('全部', 'DT'),
 ('精力', 'NN'),
 ('进行', 'VV'),
 ('系统', 'NN'),
 ('优化', 'NN'),
 ('工作', 'NN')]

In [27]:
content_parse[1]

'(ROOT\n  (IP\n    (NP\n      (NP (NN 体验) (NN 版))\n      (NP (NN 内测)))\n    (PU ，)\n    (VP\n      (VP\n        (ADVP (AD 稳))\n        (VP (VV 定版)\n          (IP\n            (VP\n              (ADVP (AD 暂))\n              (ADVP (AD 不))\n              (VP (VV 受)\n                (NP (NN 影响)))))))\n      (PU ，)\n      (VP (MSP 以)\n        (VP (VV 确保)\n          (IP\n            (NP (NN 工程师))\n            (VP (VV 可以)\n              (VP\n                (VP (VV 集中)\n                  (NP\n                    (DP (DT 全部))\n                    (NP (NN 精力))))\n                (VP (VV 进行)\n                  (NP (NN 系统) (NN 优化) (NN 工作)))))))))))'

In [26]:
content_dependency_parse[1]

[('ROOT', 0, 6),
 ('compound:nn', 2, 1),
 ('compound:nn', 3, 2),
 ('nsubj', 6, 3),
 ('punct', 6, 4),
 ('advmod', 6, 5),
 ('advmod', 9, 7),
 ('neg', 9, 8),
 ('ccomp', 6, 9),
 ('dobj', 9, 10),
 ('punct', 9, 11),
 ('aux:prtmod', 13, 12),
 ('conj', 9, 13),
 ('nsubj', 16, 14),
 ('aux:modal', 16, 15),
 ('ccomp', 13, 16),
 ('det', 18, 17),
 ('dobj', 16, 18),
 ('conj', 16, 19),
 ('compound:nn', 22, 20),
 ('compound:nn', 22, 21),
 ('dobj', 19, 22)]

## 使用哈工大的ltp进行分析

In [ ]:
/Users/lingrowzhang/Documents/GitHub/Artificial-Intelligence-for-NLP/ltp_data_v3.4.0